Import Library & Konfigurasi Path

In [ ]:
# %% [markdown]
# # FASE 2: Pembangunan Model Rekomendasi Wisata (TF-IDF + Cosine Similarity)
# 
# **Tujuan:**
# 1. Membaca dataset bersih dari `data/processed/`.
# 2. Mengubah teks ("DNA" wisata) ke dalam bentuk vektor TF-IDF.
# 3. Menghitung matriks kemiripan antar semua destinasi.
# 4. Menyimpan *artefak* model (vectorizer dan matriks) ke folder `models/` untuk digunakan di aplikasi.

# %%
# ======================================================
# 1️⃣ IMPORT LIBRARY & KONFIGURASI PATH
# ======================================================
import pandas as pd
import pickle
from pathlib import Path

# Import library scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Inisialisasi path utama
BASE_DIR = Path().resolve().parent

# Path untuk input & output
PROCESSED_DATA_PATH = BASE_DIR / "data" / "processed" / "destinasi_processed.csv"
MODELS_DIR = BASE_DIR / "models"
MODELS_DIR.mkdir(parents=True, exist_ok=True) # Membuat folder jika belum ada

print("✅ Library dan path berhasil dikonfigurasi.")

Memuat Dataset yang Sudah Diproses

In [ ]:
# %% [markdown]
# ## 2. Memuat Dataset yang Sudah Diproses
# 
# Kita akan menggunakan file `destinasi_processed.csv` yang telah kita siapkan di Fase 1.

# %%
# ======================================================
# 2️⃣ MEMUAT DATASET BERSIH
# ======================================================
try:
    df = pd.read_csv(PROCESSED_DATA_PATH)
    print(f"✅ Dataset berhasil dimuat dari '{PROCESSED_DATA_PATH.name}'")
    print(f"Jumlah baris: {len(df)}, Jumlah kolom: {len(df.columns)}\n")
except FileNotFoundError:
    raise FileNotFoundError(f"❌ File '{PROCESSED_DATA_PATH.name}' tidak ditemukan. Pastikan Anda sudah menjalankan notebook Fase 1 terlebih dahulu.")

# Pastikan kolom 'fitur_bersih' tidak kosong
df['fitur_bersih'] = df['fitur_bersih'].fillna('')

# Tampilkan beberapa contoh data untuk verifikasi
df[['id', 'nama_wisata', 'fitur_bersih']].head()

Membuat Representasi TF-IDF

In [ ]:
# %% [markdown]
# ## 3. Membuat Representasi TF-IDF
# 
# TF-IDF (Term Frequency-Inverse Document Frequency) akan mengubah setiap "DNA" teks menjadi sebuah vektor angka. Vektor ini merepresentasikan pentingnya setiap kata dalam konteks seluruh dataset.

# %%
# ======================================================
# 3️⃣ MEMBUAT VEKTOR TF-IDF
# ======================================================
# Inisialisasi TF-IDF Vectorizer
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),    # ⭐ PENYEMPURNAAN: Mempertimbangkan frasa 2 kata (misal: "pasir putih") selain kata tunggal.
    max_features=5000      # Membatasi jumlah fitur (kata/frasa) agar efisien
)

# Latih TF-IDF pada data teks dan ubah teks ke dalam bentuk matriks
tfidf_matrix = tfidf.fit_transform(df['fitur_bersih'])

print("✅ Matriks TF-IDF berhasil dibuat.")
print(f"Ukuran matriks: {tfidf_matrix.shape} (artinya ada {tfidf_matrix.shape[0]} destinasi dan {tfidf_matrix.shape[1]} fitur kata/frasa unik)")

Menghitung Cosine Similarity

In [ ]:
# %% [markdown]
# ## 4. Menghitung Cosine Similarity
# 
# Setelah memiliki representasi vektor untuk setiap destinasi, kita akan menghitung "jarak" atau kemiripan antar vektor tersebut menggunakan Cosine Similarity. Hasilnya adalah sebuah matriks di mana setiap sel `(i, j)` berisi skor kemiripan antara destinasi `i` dan `j`.

# %%
# ======================================================
# 4️⃣ MENGHITUNG MATRIKS KEMIRIPAN
# ======================================================
similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

print("✅ Matriks kemiripan (cosine similarity) berhasil dihitung.")
print(f"Dimensi matriks: {similarity_matrix.shape}")

Menyimpan Model dan Matriks

In [ ]:
# %% [markdown]
# ## 5. Menyimpan Artefak Model
# 
# Kita menyimpan `TfidfVectorizer` dan `similarity_matrix` agar bisa digunakan kembali oleh aplikasi Streamlit tanpa perlu melatih ulang. Ini adalah inti dari fase "Offline Training".

# %%
# ======================================================
# 5️⃣ MENYIMPAN MODEL & MATRIKS
# ======================================================
# Simpan TF-IDF vectorizer
with open(MODELS_DIR / "tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(tfidf, f)
print(f"💾 TF-IDF Vectorizer berhasil disimpan di: {MODELS_DIR / 'tfidf_vectorizer.pkl'}")

# Simpan similarity matrix
with open(MODELS_DIR / "similarity_matrix.pkl", "wb") as f:
    pickle.dump(similarity_matrix, f)
print(f"💾 Matriks Kemiripan berhasil disimpan di: {MODELS_DIR / 'similarity_matrix.pkl'}")

Validasi Cepat

In [ ]:
# %% [markdown]
# ## 6. Validasi Cepat
# 
# Sebelum mengakhiri, mari kita lakukan tes sederhana untuk memastikan model kita memberikan rekomendasi yang masuk akal. Kita akan memilih satu destinasi dan melihat 5 destinasi lain yang paling mirip menurut model.

# %%
# ======================================================
# 6️⃣ VALIDASI HASIL REKOMENDASI
# ======================================================
# Membuat DataFrame dari matriks kemiripan untuk kemudahan pembacaan
cosine_sim_df = pd.DataFrame(similarity_matrix, index=df['nama_wisata'], columns=df['nama_wisata'])

# --- Uji Coba Rekomendasi ---
# Pilih satu nama wisata sebagai contoh
nama_wisata_referensi = 'Pantai Papuma'

print(f"🎯 5 Destinasi paling mirip dengan '{nama_wisata_referensi}':\n")

# Ambil 5 wisata paling mirip (skor tertinggi), dan hilangkan wisata itu sendiri dari hasil
top_5_similar = cosine_sim_df[nama_wisata_referensi].sort_values(ascending=False)[1:6]

# Tampilkan hasilnya
display(top_5_similar)